# Importing Required Libraries

# Opening the Webcam and Checking 


# Setting the Camera Resolution And  Loading Stored Faces for Recognition

# Identifying the Attendance Column for Today

# Capturing Video and Detecting Faces And Marking Attendance in the Excel File And  Drawing Boxes Around Detected Faces

# Closing the Camera and Saving Data

In [2]:
import cv2
import face_recognition
import openpyxl
from datetime import datetime
import numpy as np
import os

# بدء التقاط الفيديو من الكاميرا
video_cap = cv2.VideoCapture(0)

# التحقق من أن الكاميرا تعمل
if not video_cap.isOpened():
    print("⚠️ خطأ: تعذر فتح الكاميرا.")
    exit()

# ضبط دقة الفيديو
video_cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
video_cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

print("📷 الكاميرا تعمل... اضغط 'q' للإغلاق")

# 📌 تحميل جميع الصور من مجلد 'face_id_photos/' والتعرف على الوجوه
face_encodings_list = []
face_names = []

photo_dir = "photo/"
if not os.path.exists(photo_dir):
    print(f"⚠️ خطأ: المجلد '{photo_dir}' غير موجود.")
    exit()

for file_name in os.listdir(photo_dir):
    if file_name.endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join(photo_dir, file_name)
        image = face_recognition.load_image_file(image_path)
        encodings = face_recognition.face_encodings(image)

        if encodings:
            face_encodings_list.append(encodings[0])
            face_names.append(os.path.splitext(file_name)[0])
        else:
            print(f"⚠️ تحذير: لم يتم العثور على وجه في {file_name}")

if not face_encodings_list:
    print("⚠️ خطأ: لم يتم تحميل أي وجوه، تأكد من وضع صور صالحة في المجلد.")
    exit()

print(f"تم تحميل الأسماء التالية: {face_names}")

# فتح ملف Excel الموجود
excel_file = 'Attendance.xlsx'
if not os.path.exists(excel_file):
    print(f"⚠️ خطأ: الملف {excel_file} غير موجود.")
    exit()

wb = openpyxl.load_workbook(excel_file)
ws = wb.active

# تحديد عمود الحضور لهذا اليوم
current_date = datetime.now().strftime("%Y-%m-%d")
attendance_column = None

# البحث عن عمود بتاريخ اليوم
for col in range(2, ws.max_column + 1):
    if ws.cell(row=1, column=col).value == current_date:
        attendance_column = col
        break

# إذا لم يكن موجودًا، يتم إنشاء عمود جديد
if not attendance_column:
    attendance_column = ws.max_column + 1
    ws.cell(row=1, column=attendance_column).value = current_date

# قائمة الطلاب المفترض تسجيل حضورهم (مستمدة من أسماء الوجوه)
student_list = face_names.copy()

try:
    while True:
        ret, frame = video_cap.read()
        if not ret:
            print("⚠️ خطأ: لم يتم التقاط صورة من الكاميرا.")
            break

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb)
        print(f"✅ تم اكتشاف {len(face_locations)} وجه(وجوه)")

        face_names_detected = []
        if face_locations:
            face_encodings = face_recognition.face_encodings(rgb, face_locations)
            for face_encoding in face_encodings:
                matches = face_recognition.compare_faces(face_encodings_list, face_encoding, tolerance=0.5)
                face_distances = face_recognition.face_distance(face_encodings_list, face_encoding)
                print(f"🔍 المسافات بين الوجوه: {face_distances}")

                if any(matches):
                    best_match = np.argmin(face_distances)
                    name = face_names[best_match]
                else:
                    name = "Unknown"
                
                face_names_detected.append(name)
                
                # تسجيل الحضور في ملف Excel إذا كان الاسم معروفًا ولم يتم تسجيله مسبقًا
                if name in face_names and name in student_list:
                    student_list.remove(name)
                    
                    # البحث عن الصف المناسب للاسم في ملف Excel
                    for row in range(2, ws.max_row + 1):
                        if ws.cell(row=row, column=1).value == name:
                            # التحقق مما إذا كان الحضور مسجلًا بالفعل في عمود اليوم
                            if ws.cell(row=row, column=attendance_column).value is None:
                                ws.cell(row=row, column=attendance_column).value = True
                                print(f"📝 تم تسجيل الحضور للطالب {name} في ملف Excel")
                            else:
                                print(f"✅ {name} مسجل بالفعل اليوم، لم يتم التغيير")
                            break

        # رسم مستطيل حول الوجوه وكتابة الاسم
        for (top, right, bottom, left), name in zip(face_locations, face_names_detected):
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        cv2.imshow('Face Recognition', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("🔴 إغلاق الكاميرا.")
            break
except KeyboardInterrupt:
    print("🔴 تم إغلاق البرنامج.")
finally:
    # التحقق من الطلاب الذين لم يحضروا درس اليوم
    if student_list:
        for absent_student in student_list:
            print(f"🔴 الطالب {absent_student} لم يحضر درس اليوم.")
            # يمكنك هنا إضافة الكود لإرسال رسالة تنبيه للطالب (مثلاً عبر البريد الإلكتروني)

    # التحقق من عدد الحضور خلال الشهر لكل طالب
    for row in range(2, ws.max_row + 1):
        student_name = ws.cell(row=row, column=1).value
        attendance_count = 0
        # نفترض أن الأعمدة من 2 حتى آخر عمود تمثل الدروس في هذا الشهر
        for col in range(2, ws.max_column + 1):
            if ws.cell(row=row, column=col).value == True:
                attendance_count += 1
        if attendance_count < 6:
            print(f"🔔 الطالب {student_name} لم يكمل حضور الدروس المطلوب هذا الشهر ({attendance_count}/6).")
            # يمكنك هنا إضافة الكود لإرسال رسالة تنبيه للطالب

    wb.save(excel_file)
    video_cap.release()
    cv2.destroyAllWindows()
    print("✅ تم إغلاق الكاميرا وإيقاف البرنامج بنجاح.")


📷 الكاميرا تعمل... اضغط 'q' للإغلاق
تم تحميل الأسماء التالية: ['eman', 'kareem', 'mahoud', 'mohamed', 'mostafa', 'nada', 'youisf']
✅ تم اكتشاف 1 وجه(وجوه)
🔍 المسافات بين الوجوه: [0.789184   0.38827478 0.61576336 0.64127109 0.65972947 0.74285169
 0.6415747 ]
📝 تم تسجيل الحضور للطالب kareem في ملف Excel
✅ تم اكتشاف 1 وجه(وجوه)
🔍 المسافات بين الوجوه: [0.79401931 0.39067163 0.62237242 0.63972602 0.67292461 0.75715364
 0.63224882]
✅ تم اكتشاف 1 وجه(وجوه)
🔍 المسافات بين الوجوه: [0.79124651 0.35611407 0.61809494 0.64021339 0.69969764 0.72795133
 0.63411332]
✅ تم اكتشاف 1 وجه(وجوه)
🔍 المسافات بين الوجوه: [0.81003007 0.35041894 0.61567755 0.62771409 0.70358528 0.73970752
 0.65227404]
🔴 إغلاق الكاميرا.
🔴 الطالب eman لم يحضر درس اليوم.
🔴 الطالب mahoud لم يحضر درس اليوم.
🔴 الطالب mohamed لم يحضر درس اليوم.
🔴 الطالب mostafa لم يحضر درس اليوم.
🔴 الطالب nada لم يحضر درس اليوم.
🔴 الطالب youisf لم يحضر درس اليوم.
🔔 الطالب nada لم يكمل حضور الدروس المطلوب هذا الشهر (1/6).
🔔 الطالب youisf لم يكمل حضور ال